In [1]:
# Install required libraries
!pip install torch torchvision torch-geometric scikit-learn

import torch
import torch.nn as nn
import torch.optim as optim
import torchvision.models as models
import torchvision.transforms as transforms
import torchvision.datasets as datasets
from torch.utils.data import DataLoader
import numpy as np
from sklearn.preprocessing import LabelEncoder
from sklearn.neighbors import kneighbors_graph
from torch_geometric.data import Data
from torch_geometric.nn import GCNConv, BatchNorm
import torch.nn.functional as F
import os

# Check for GPU
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

# Auto-download EuroSAT dataset
data_dir = './eurosat'
transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.RandomHorizontalFlip(),
    transforms.RandomRotation(15),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
])

# Load dataset
dataset = datasets.EuroSAT(root=data_dir, download=True, transform=transform)
dataloader = DataLoader(dataset, batch_size=32, shuffle=True, num_workers=4, pin_memory=True)

print(f"Dataset loaded: {len(dataset)} images.")

# Fine-tune ResNet101
resnet = models.resnet101(pretrained=True)
resnet.fc = nn.Linear(resnet.fc.in_features, len(set(dataset.targets)))  # Fix class count
resnet.to(device)
optimizer = optim.AdamW(resnet.parameters(), lr=0.0003, weight_decay=1e-4)
criterion = nn.CrossEntropyLoss()

# Train ResNet101
def train_resnet(model, dataloader, optimizer, criterion, epochs=15):
    model.train()
    scaler = torch.cuda.amp.GradScaler()
    for epoch in range(epochs):
        total_loss, correct = 0, 0
        for img, label in dataloader:
            img, label = img.to(device), label.to(device)
            optimizer.zero_grad()
            with torch.cuda.amp.autocast():
                output = model(img)
                loss = criterion(output, label)
            scaler.scale(loss).backward()
            scaler.step(optimizer)
            scaler.update()
            total_loss += loss.item()
            correct += (output.argmax(dim=1) == label).sum().item()
        accuracy = correct / len(dataset)
        print(f"Epoch {epoch+1}, Loss: {total_loss/len(dataloader):.4f}, Accuracy: {accuracy:.4f}")
    print(f"Final ResNet Accuracy: {accuracy:.4f}")

train_resnet(resnet, dataloader, optimizer, criterion, epochs=15)

# Feature Extraction
resnet.fc = nn.Identity()
def extract_features(dataset, model):
    model.eval()
    features, labels = [], []
    with torch.no_grad():
        for img, label in dataset:
            img = img.unsqueeze(0).to(device)
            feat = model(img).cpu().numpy().flatten()
            features.append(feat)
            labels.append(label)
    return np.array(features), np.array(labels)

features, labels = extract_features(dataset, resnet)
labels = LabelEncoder().fit_transform(labels)

# Build Optimized KNN Graph
def build_knn_graph(features, k=10):
    adj_matrix = kneighbors_graph(features, k, mode='connectivity', include_self=True).toarray()
    edge_index = np.array(np.nonzero(adj_matrix))
    return torch.tensor(edge_index, dtype=torch.long)

edge_index = build_knn_graph(features)

graph_data = Data(
    x=torch.tensor(features, dtype=torch.float),
    edge_index=edge_index,
    y=torch.tensor(labels, dtype=torch.long)
)

# Define Improved GCN Model
class GCN(nn.Module):
    def __init__(self, in_channels, hidden_channels, out_channels):
        super(GCN, self).__init__()
        self.conv1 = GCNConv(in_channels, hidden_channels)
        self.norm1 = BatchNorm(hidden_channels)
        self.conv2 = GCNConv(hidden_channels, hidden_channels)
        self.norm2 = BatchNorm(hidden_channels)
        self.conv3 = GCNConv(hidden_channels, out_channels)
        self.dropout = nn.Dropout(0.5)

    def forward(self, x, edge_index):
        x = self.conv1(x, edge_index).relu()
        x = self.norm1(x)
        x = self.dropout(x)
        x = self.conv2(x, edge_index).relu()
        x = self.norm2(x)
        x = self.dropout(x)
        x = self.conv3(x, edge_index)
        return F.log_softmax(x, dim=1)

# Initialize & Train GCN
gcn = GCN(in_channels=features.shape[1], hidden_channels=512, out_channels=len(set(labels))).to(device)
optimizer = optim.AdamW(gcn.parameters(), lr=0.001, weight_decay=1e-4)
scheduler = optim.lr_scheduler.CosineAnnealingLR(optimizer, T_max=50)
loss_fn = nn.CrossEntropyLoss()

def train_gcn(model, data, optimizer, loss_fn, scheduler, epochs=100):
    model.train()
    best_acc = 0.0
    for epoch in range(epochs):
        optimizer.zero_grad()
        out = model(data.x.to(device), data.edge_index.to(device))
        loss = loss_fn(out, data.y.to(device))
        loss.backward()
        optimizer.step()
        scheduler.step()
        acc = (out.argmax(dim=1) == data.y.to(device)).float().mean().item()
        best_acc = max(best_acc, acc)
        print(f"Epoch {epoch+1}, Loss: {loss.item():.4f}, Accuracy: {acc:.4f}")
    print(f"Final GCN Accuracy: {best_acc:.4f}")

train_gcn(gcn, graph_data, optimizer, loss_fn, scheduler, epochs=100)

print("Training complete. Best accuracy achieved during training is displayed above.")

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 63.1/63.1 kB 2.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 3.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 31.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 20.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 23.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 12.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 5.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 63.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 39.5 MB/s eta 0:00:00
  Attempting uninstall: nvidia-nvjitlink-cu12


100%|██████████| 94.3M/94.3M [00:00<00:00, 301MB/s]


Extracting ./eurosat/eurosat/EuroSAT.zip to ./eurosat/eurosat


/usr/local/lib/python3.11/dist-packages/torch/utils/data/dataloader.py:617: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet101_Weights.IMAGENET1K_V1`. You can also use `weights=ResN

Dataset loaded: 27000 images.


Downloading: "https://download.pytorch.org/models/resnet101-63fe2227.pth" to /root/.cache/torch/hub/checkpoints/resnet101-63fe2227.pth
100%|██████████| 171M/171M [00:01<00:00, 163MB/s]
<ipython-input-1-c44d8010d231>:49: FutureWarning: `torch.cuda.amp.GradScaler(args...)` is deprecated. Please use `torch.amp.GradScaler('cuda', args...)` instead.
  scaler = torch.cuda.amp.GradScaler()
<ipython-input-1-c44d8010d231>:55: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with torch.cuda.amp.autocast():


Epoch 1, Loss: 0.3163, Accuracy: 0.8982
Epoch 2, Loss: 0.1639, Accuracy: 0.9469
Epoch 3, Loss: 0.1341, Accuracy: 0.9563
Epoch 4, Loss: 0.1062, Accuracy: 0.9650
Epoch 5, Loss: 0.0971, Accuracy: 0.9677
Epoch 6, Loss: 0.0943, Accuracy: 0.9677
Epoch 7, Loss: 0.0778, Accuracy: 0.9741
Epoch 8, Loss: 0.0737, Accuracy: 0.9755
Epoch 9, Loss: 0.0710, Accuracy: 0.9769
Epoch 10, Loss: 0.0646, Accuracy: 0.9790
Epoch 11, Loss: 0.0604, Accuracy: 0.9793
Epoch 12, Loss: 0.0589, Accuracy: 0.9801
Epoch 13, Loss: 0.0515, Accuracy: 0.9828
Epoch 14, Loss: 0.0525, Accuracy: 0.9820
Epoch 15, Loss: 0.0478, Accuracy: 0.9833
Final ResNet Accuracy: 0.9833
Epoch 1, Loss: 2.9315, Accuracy: 0.0869
Epoch 2, Loss: 0.1352, Accuracy: 0.9660
Epoch 3, Loss: 0.0672, Accuracy: 0.9839
Epoch 4, Loss: 0.0541, Accuracy: 0.9856
Epoch 5, Loss: 0.0477, Accuracy: 0.9878
Epoch 6, Loss: 0.0418, Accuracy: 0.9883
Epoch 7, Loss: 0.0402, Accuracy: 0.9898
Epoch 8, Loss: 0.0387, Accuracy: 0.9899
Epoch 9, Loss: 0.0396, Accuracy: 0.9892
Epoc